# Declustering applied to the Rott 2021 sequence

In [ ]:
%matplotlib inline

Import python modules

In [ ]:
import datetime
import matplotlib
import numpy as np

## ROB-specific modules
import mapping.layeredbasemap as lbm
import eqcatalog

Read earthquake sequence from database

In [ ]:
start_date = datetime.date(2021, 1, 1)
cat = eqcatalog.rob.query_local_eq_catalog(start_date=start_date,
                                           region=(6.0, 6.4, 50.58, 50.875))
cat.name = 'Rott sequence'
cat.print_info()

Since 1 January 2021, a seismic sequence is occurring in the border region of Belgium and Germany. After 2 months, 129 events have occurred, ranging in magnitude ($M_L$) between -1.4 and 2.6. The mainshock ($M_L=2.6$) occurred on the 2nd day, but another $M_L=2.6$ earthquake occurred on day 14.

In [ ]:
cat.subselect(Mmin=2.6, Mtype='ML').print_list()

The map shows 2 spatial clusters: one around Rott in the SW, and the other around Eschweiler in the NE.

In [ ]:
## Color is by age (blue = oldest, red = most recent)
color = lbm.ThematicStyleColormap('jet', value_key='year', add_legend=False)
cat.plot_map(resolution='h', edge_color=color)

In [ ]:
cat.plot_time_magnitude(Mtype='ML', xtick_rotation=35)

Plot magnitude-frequency distribution. Completeness magnitude can be estimated as:
* $M_L:$ 0.8 (0.2 if you are optimistic)
* $M_W:$ 1.0 (0.4 if you are optimistic)

In [ ]:
## Note: OpenQuake MFDs do not support negative magnitudes...
min_mag, max_mag = 0., 3
bin_width = 0.2
for Mtype in ('MW', 'ML'):
    Mrelation = {}
    if Mtype == 'MW':
        Mrelation = {'ML': 'GruenthalEtAl2009'}
    completeness = cat.get_uniform_completeness(min_mag, Mtype=Mtype)
    seq_mfd = cat.get_incremental_mfd(min_mag, max_mag, bin_width, Mtype=Mtype,
                                      Mrelation=Mrelation, completeness=completeness)
    seq_mfd.plot()

## Declustering analysis

Check if different declustering windows/methods identify sequence as 1 cluster. Note that all declustering methods are based on $M_W$, so we need a conversion from $M_L$.

In [ ]:
Mrelation = {'ML': 'GruenthalEtAl2009'}

### Linked-window method

First, we test the linked-window method. This is a variant of the original windowing method of Gardner & Knopoff, but taking into account the combined space/time windows of all earthquakes in the cluster rather than only the mainshock. Ignoring or taking into account location errors does not change much.

In [ ]:
dc_method = eqcatalog.declustering.LinkedWindowMethod()

In [ ]:
lw_dc_results = {}
for dc_window_name in ('GardnerKnopoff1974', 'Uhrhammer1986', 'Gruenthal2009'):
    print(dc_window_name)
    dc_window = eqcatalog.declustering.get_window_by_name(dc_window_name)
    dc_result = dc_method.analyze_clusters(cat, dc_window, Mrelation,
                                          ignore_location_errors=False)
    dc_result.print_info()
    lw_dc_results[dc_window_name] = dc_result
    print()

A map of the clusters obtained with the Grünthal window definitions shows that the identified clusters overlap both areas.

In [ ]:
dc_result = lw_dc_results['Gruenthal2009']
catalogs = [cluster.to_catalog() for cluster in dc_result.get_clusters()]
catalogs.append(dc_result.get_unclustered_events())
labels = ['Cluster #%d' % i for i in range(dc_result.get_num_clusters())]
labels.append('Unclustered')
cm = matplotlib.cm.rainbow
colors = [cm(i) for i in np.linspace(0, 1, len(catalogs))]
eqcatalog.plot.plot_map(catalogs, labels=labels, edge_colors=colors, resolution='h')

In [ ]:
dc_result.get_unclustered_events().print_list()

### Reasenberg method

Now we test the Reasenberg declustering method. Note the strong sensitivity to location errors! On the other hand, the $xmeff$ parameter does not seem to have a strong influence.

In [ ]:
## For the window definition, it is possible to adjust:
## - rfact: number of crack radii (default: 10)
## - dsigma: stress drop in bars (default: 30)
## - rmax: max. interaction distance in km (default: 30)
## - tau_min: min. length of time window in minutes (default: 2880 = 2 days)
## - tau_max: max. length of time window in minutes (default: 144000 = 10 days)
## - xmeff: "effective" lower magnitude cutoff (default: 1.5)
## - xk: factor used to raise xmeff (default: 0.5)
## - p1: confidence level (default: 0.99)
dc_method = eqcatalog.declustering.ReasenbergMethod()
for ignore_location_errors in (True, False):
    if ignore_location_errors:
        xmeff_values = [1.0]
    else:
        xmeff_values = [0.4, 1.0, 1.5]
    for xmeff in xmeff_values:
        dc_window = eqcatalog.declustering.Reasenberg1985Window(dsigma=30, xmeff=xmeff)
        dc_result = dc_method.analyze_clusters(cat, Mrelation, dc_window,
                                               ignore_location_errors=ignore_location_errors)
        print('Reasenberg1985 (ignore_location_errors=%s, xmeff=%.1f)'
              % (ignore_location_errors, xmeff))
        dc_result.print_info()
        print()

In [ ]:
dc_result.get_unclustered_events().print_list()

When location errors are taken into account, the Reasenberg method identifies two main clusters that are separated in space.

In [ ]:
catalogs = [cluster.to_catalog() for cluster in dc_result.get_clusters()]
catalogs.append(dc_result.get_unclustered_events())
labels = ['Cluster #%d' % i for i in range(dc_result.get_num_clusters())]
labels.append('Unclustered')
cm = matplotlib.cm.rainbow
colors = [cm(i) for i in np.linspace(0, 1, len(catalogs))]
eqcatalog.plot.plot_map(catalogs, labels=labels, edge_colors=colors, resolution='h')

## Omori-law fitting

We use the optimistic value for the completeness magnitude Mc

In [ ]:
Mc = 0.2
#Mc = 0.8

Isolate mainshock from catalog

In [ ]:
mainshock = cat.get_event_by_id(11630)
Mm = mainshock.ML
mainshock.print_info()

Isolate aftershocks from cluster

In [ ]:
cluster = dc_result.get_cluster_by_eq(mainshock)
aftershocks = cluster.get_aftershocks()
aftershocks.print_info()

Apply completeness magnitude constraint

In [ ]:
cc_aftershocks = aftershocks.subselect(Mmin=Mc, Mtype='ML')
print(len(cc_aftershocks))

Determine elapsed times (in days)

In [ ]:
as_time_deltas = cc_aftershocks.get_time_deltas(mainshock.datetime)
as_time_deltas = eqcatalog.time.fractional_time_delta(as_time_deltas, 'D')
as_time_deltas

Estimate parameters K, c, p of Omori law based on elapsed times since mainshock

In [ ]:
#(K1, c1, p1) = eqcatalog.omori.estimate_omori_params(as_time_deltas)
(K1, c1, p1), _, _ = eqcatalog.omori.OmoriLaw.fit_cumulative(as_time_deltas,
                                                             np.arange(len(as_time_deltas)))
print(K1, c1, p1)

Define Omori law. Note that value of K depends on completeness magnitude and mainshock magnitude, so these are inherent properties of the Omori law! Default time unit is days.

In [ ]:
omlaw = eqcatalog.omori.OmoriLaw(K1, c1, p1, Mc, Mm)

Plot cumulative number of aftershocks versus time. Different colors correspond to different clusters identified with the linked-window method (Grünthal window).

In [ ]:
today = datetime.date.today()
num_days = (today - start_date).days
x_values = np.linspace(0, num_days, 100)
marker_sizes = ((np.array([eq.ML for eq in cc_aftershocks]) - Mc) + 2)**2
observed_cluster_idxs = [lw_dc_results['Gruenthal2009'].get_cluster_by_eq(eq).ID
                         for eq in cc_aftershocks]
unique_cluster_idxs = np.unique(observed_cluster_idxs)
cm = matplotlib.cm.rainbow
colors = [cm(i) for i in np.linspace(0, 1, len(unique_cluster_idxs))]
label = 'Omori fit ($M_c=%.1f$)' % Mc
omlaw.plot_cumulative(x_values, observed_delta_t=as_time_deltas, xscaling='lin',
                      observed_marker_sizes=marker_sizes, observed_marker='o',
                      observed_cluster_colors=colors,
                      observed_cluster_idxs=observed_cluster_idxs, label=label)